<h1>Proton Therapy Optimization</h1>

<h3>Naive modeling</h3>

The first idea that comes to mind is to make a straight forward mathematical translation of the oral problem description. That would be to set a vector of *i* integer variables *x[i]*, representing the number of proton fractions that should be performed to patient *i*. 

We would then symbolize our input matrix as *c[i,j]*: the benefit of offering *j* proton fractions to patient *i*. While this is a very intuitive model it suffers from a very non linear object function. In the simplest case where we would like to optimize the sum or average BED, the object functions to be maximized would be:

**f = Σ c[i, x[i]] over all i's**

<h3>A better modeling idea</h3>


Instead we would prefer to go with a linear model. For that reason we could select another set of decision variables, that is a binary matrix *x[i,j]* where a value of 1 denotes that *j* fractions should be performed on patient *i*. Obviously we should constraint every rows sum to 1 (a patient will receive 0,1, ... or N fractions).

Our object function can now be expressed as elegandly as:

**f = Σ c[i,j] * x[i,j] over all j's over all i's**

This is obviously as linear as it gets. Lets code it!

In [2]:
# GUROBI must be installed and a licence file must be discoverable in one of the default locations
from gurobipy import *

# Get some additional dependencies
import pandas as pd
import numpy as np

In [34]:
# Read the dataset into a 2d array. benefit[i,j] is the benefit patient i will get from j fractions
data = pd.read_csv('PayoffMatrix.txt', delim_whitespace = True)
benefit = data.values

(16, 16)

In [53]:
# Lets represent our decisions as indexes
num_patients, max_fractions_per_patient = benefit.shape
patients = [i for i in range(num_patients)]
fractions = [j for j in range(max_fractions_per_patient)]

In [60]:
m = Model('proton_therapy')

x = m.addVars(num_patients, max_fractions_per_patient, vtype = GRB.BINARY)

# Only one choice of fractions per patient is valid
m.addConstrs(quicksum(x[i,j] for j in fractions) == 1 for i in patients)

# We can only perform so many proton therapies per week
CAPACITY = 100
m.addConstr(quicksum(quicksum(x[i,j] * fractions[j] for j in fractions) for i in patients) <= CAPACITY)

m.setObjective(quicksum(x[i,j] * benefit[i,j] for i in patients for j in fractions), GRB.MAXIMIZE)
m.update()
m.optimize()

Optimize a model with 17 rows, 256 columns and 496 nonzeros
Variable types: 0 continuous, 256 integer (256 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [4e+01, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective 1154.63
Presolve removed 0 rows and 37 columns
Presolve time: 0.00s
Presolved: 17 rows, 219 columns, 425 nonzeros
Variable types: 0 continuous, 219 integer (219 binary)

Root relaxation: objective 1.637214e+03, 54 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    1637.2142020 1637.21420  0.00%     -    0s

Explored 0 nodes (54 simplex iterations) in 0.02 seconds
Thread count was 8 (of 8 available processors)

Solution count 2: 1637.21 1154.63 
Pool objective bound 1637.21

Optimal solution found (tolerance 1.00e-04)
Best obj

In [61]:
x

{(0, 0): <gurobi.Var C0 (value -0.0)>,
 (0, 1): <gurobi.Var C1 (value -0.0)>,
 (0, 2): <gurobi.Var C2 (value -0.0)>,
 (0, 3): <gurobi.Var C3 (value -0.0)>,
 (0, 4): <gurobi.Var C4 (value -0.0)>,
 (0, 5): <gurobi.Var C5 (value -0.0)>,
 (0, 6): <gurobi.Var C6 (value -0.0)>,
 (0, 7): <gurobi.Var C7 (value -0.0)>,
 (0, 8): <gurobi.Var C8 (value -0.0)>,
 (0, 9): <gurobi.Var C9 (value 0.0)>,
 (0, 10): <gurobi.Var C10 (value 1.0)>,
 (0, 11): <gurobi.Var C11 (value -0.0)>,
 (0, 12): <gurobi.Var C12 (value -0.0)>,
 (0, 13): <gurobi.Var C13 (value -0.0)>,
 (0, 14): <gurobi.Var C14 (value -0.0)>,
 (0, 15): <gurobi.Var C15 (value -0.0)>,
 (1, 0): <gurobi.Var C16 (value -0.0)>,
 (1, 1): <gurobi.Var C17 (value -0.0)>,
 (1, 2): <gurobi.Var C18 (value 0.0)>,
 (1, 3): <gurobi.Var C19 (value 1.0)>,
 (1, 4): <gurobi.Var C20 (value -0.0)>,
 (1, 5): <gurobi.Var C21 (value -0.0)>,
 (1, 6): <gurobi.Var C22 (value -0.0)>,
 (1, 7): <gurobi.Var C23 (value -0.0)>,
 (1, 8): <gurobi.Var C24 (value -0.0)>,
 (1, 9):